In [3]:
import pandas as pd

In [5]:
df_listing = pd.read_csv("Data/listings.csv")

In [7]:
df_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6893 entries, 0 to 6892
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6893 non-null   int64  
 1   listing_url                                   6893 non-null   object 
 2   scrape_id                                     6893 non-null   int64  
 3   last_scraped                                  6893 non-null   object 
 4   source                                        6893 non-null   object 
 5   name                                          6893 non-null   object 
 6   description                                   6877 non-null   object 
 7   neighborhood_overview                         4628 non-null   object 
 8   picture_url                                   6893 non-null   object 
 9   host_id                                       6893 non-null   i

### Calculez le nombre d’host et le nombre de review par quartier. (​​neighbourhood_cleansed)?

In [15]:
nombre_host = df_listing.groupby('neighbourhood_cleansed').host_id.nunique()

In [18]:
nombre_review = df_listing.groupby('neighbourhood_cleansed').number_of_reviews.sum()

In [22]:
pd.concat([nombre_host,nombre_review], axis = 1)

,host_id,number_of_reviews
neighbourhood_cleansed,,
Bijlmer-Centrum,76,4964
Bijlmer-Oost,29,983
Bos en Lommer,284,10586
Buitenveldert - Zuidas,111,5691
Centrum-Oost,489,45227
Centrum-West,674,69068
De Aker - Nieuw Sloten,69,6330
De Baarsjes - Oud-West,916,40372
De Pijp - Rivierenbuurt,644,30418


### Quels sont les pourcentage d’acceptation et de réponse moyen des hosts?

In [129]:
df_listing["host_response_rate"].dropna().apply(lambda x : x.replace("%","")).astype("int").mean()

94.96734148204736

In [ ]:
df_listing["host_acceptance_rate"].dropna().apply(lambda x : x.replace("%","")).astype("int").mean()

### Quel pourcentage des hosts on une vérification:

- par téléphone
- par email
- par email pro

In [72]:
verification_dict = {}
for i in df_listing[df_listing["host_verifications"].notnull()]["host_verifications"].apply(eval):
    for j in i:
        if j not in verification_dict:
            verification_dict[j] = 1
        else:
            verification_dict[j] += 1

for key in verification_dict.keys():
    verification_dict[key] =  verification_dict[key] / df_listing.shape[0] 

verification_dict



{'email': 0.9281880168286668,
 'phone': 0.9968083563034963,
 'work_email': 0.1302770927027419}

### En fonction du type de room, calculez le nombre moyen d’amenities et l'écart type.

In [100]:
df_listing["nb_amenities"] = df_listing["amenities"].apply(eval).apply(len)

In [101]:
df_listing[["nb_amenities","room_type"]].groupby("room_type").agg(["mean","std"])

nb_amenities           
                        mean        std
room_type                              
Entire home/apt    30.035961  13.892002
Hotel room         22.573333   9.434229
Private room       25.735342  12.106804
Shared room        13.648649   8.018091

In [132]:
df_room_type = df_listing[["nb_amenities","room_type"]].groupby("room_type").agg(["mean","std"])
df_room_type.columns = ["nb_aminities_mean","nb_aminities_std"]
df_room_type

,nb_aminities_mean,nb_aminities_std
room_type,,
Entire home/apt,30.035961,13.892002
Hotel room,22.573333,9.434229
Private room,25.735342,12.106804
Shared room,13.648649,8.018091


### Quel est le prix median par type de room, le maximum, le minimum, le premier et le troisième quartile?

In [133]:
df_listing["clean_price"] = df_listing["price"].apply(lambda x: x.replace("$","")) \
                                               .apply(lambda x: x.replace(",","")) \
                                               .astype("float")
           

In [134]:
df_listing[["clean_price","room_type"]].groupby("room_type").agg(
    median_price=('clean_price', 'median'), 
    max_price=('clean_price', 'max'),
    min_price=('clean_price', 'min'),
    first_quartile_price=('clean_price',lambda x:x.quantile(0.25)),
    third_quartile_price=('clean_price', lambda x:x.quantile(0.75)),
    )

,median_price,max_price,min_price,first_quartile_price,third_quartile_price
room_type,,,,,
Entire home/apt,200.0,7900.0,15.0,143.0,285.0
Hotel room,222.0,999.0,0.0,114.5,271.0
Private room,130.0,1657.0,25.0,94.0,198.0
Shared room,75.0,500.0,45.0,66.0,94.0


### Si on considère qu’un "bath" vaut 1,  qu’une bath shared vaut 0.5 et qu’un private bath vaut 2. Combien y-a-t-il d’annonces par valeur de salle de bain. (0.5 / 1 / 1.5 ….)?

In [109]:
def valeur_salle_de_bain(x):
    x_list = x.split(" ")
    x_value = 1
    mapper = {"bath":1, "baths":1, "private":2, "shared":0.5, "half-bath":0.5}
    for element in x_list:
        try: 
            x_value *= float(element)
        except:
            x_value *= mapper[element.lower()]
    return x_value

valeur_salle_de_bain("1.5 shared baths")

0.75

In [135]:
df_listing["bathrooms_text"].dropna().apply(valeur_salle_de_bain).value_counts().sort_index(ascending=True)

0.00       33
0.25       17
0.50      355
0.75      198
1.00     2807
1.50     1928
1.75        1
2.00     1260
2.50      157
3.00       79
3.50       25
4.00        4
4.50        7
5.00        4
5.50        2
17.00       1
Name: bathrooms_text, dtype: int64

In [105]:
df_listing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6893 entries, 0 to 6892
Data columns (total 77 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6893 non-null   int64  
 1   listing_url                                   6893 non-null   object 
 2   scrape_id                                     6893 non-null   int64  
 3   last_scraped                                  6893 non-null   object 
 4   source                                        6893 non-null   object 
 5   name                                          6893 non-null   object 
 6   description                                   6877 non-null   object 
 7   neighborhood_overview                         4628 non-null   object 
 8   picture_url                                   6893 non-null   object 
 9   host_id                                       6893 non-null   i

In [104]:
df_listing[["bathrooms","bathrooms_text"]]

,bathrooms,bathrooms_text
0,NaN,1.5 shared baths
1,NaN,1 private bath
2,NaN,1.5 baths
3,NaN,1 shared bath
4,NaN,1 shared bath
...,...,...
6888,NaN,2.5 baths
6889,NaN,3 baths
6890,NaN,2 baths
6891,NaN,2 baths


### Il y a t’il une corrélation entre le nombre de reviews et la taille de la description en terme de nombre de caractères?

In [120]:
df_cor = df_listing[["description", "number_of_reviews"]].dropna()
df_cor["description_len"] = df_cor["description"].apply(len) 
df_cor[["description_len", "number_of_reviews"]].corr()

,description_len,number_of_reviews
description_len,1.000000,0.189901
number_of_reviews,0.189901,1.000000


### On suspecte des faux commentaires. Pour identifier cela on regarde les séjours où le nom de l’host est le même que le nom du reviewer.  Quel est ce pourcentage général de séjour où le nom de l'host est identique au nom du reviewer?

In [121]:
df_reviews = pd.read_csv("Data/reviews.csv")

In [123]:
df_merge = df_reviews.merge(df_listing, how="left", left_on="listing_id", right_on="id")

In [127]:
df_merge.loc[ df_merge["host_name"].apply(lambda x:x.lower())== df_merge["reviewer_name"].apply(lambda x:x.lower()), ["host_name","reviewer_name"]]

,host_name,reviewer_name
293,Daniel,Daniel
298,Daniel,Daniel
358,Nathalie,Nathalie
719,Daniel,Daniel
3882,Peter,Peter
...,...,...
335932,Jonathan,Jonathan
336900,Caroline,Caroline
338512,David,David
341760,Christina,Christina


In [128]:
171/df_merge.shape[0]

0.0004986818468142687